In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
from pickle import Pickler, Unpickler

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = "gdrive/My Drive/Colab Notebooks/"
df = Unpickler(open(root_path + "dataset/imoves_df", 'rb')).load()

## Normalizando dados

In [0]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))

X = df.drop(['SalePrice'], axis=1)
y = df['SalePrice'].tolist()

X = sc.fit_transform(X)

## Separando teste de treino

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

# Modelagem

## SGD

In [0]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [7]:
sgd = SGDRegressor()

sgd.fit(X_train, y_train)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [19]:
## testando e avaliando

predict = sgd.predict(X_test)
print(r2_score(y_test, predict))
print(mean_squared_error(y_test, predict))

0.8396181155583652
1177426180.835401


### Refinando hiperparâmetros

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
params = {'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
         'penalty': ['none', 'l1', 'l2', 'elasticnet'],
         'alpha': [1e-4, 1e-2, 1],
         'fit_intercept': [True, False],
         'tol': [1e-3, 1e-2, 1],
         'learning_rate':  ['constant', 'optimal', 'invscaling', 'adaptive'],
        }

In [0]:
# gs = GridSearchCV(sgd, params, scoring='r2')

In [0]:
gs.fit(X_train, y_train)

In [0]:
print(gs.best_score_)
print(gs.best_params_)

0.8187601203468228
{'alpha': 0.01, 'fit_intercept': False, 'learning_rate': 'invscaling', 'loss': 'squared_loss', 'penalty': 'l2', 'tol': 0.001}


### Testando novos hiperparâmetros

In [20]:
sgd2 = SGDRegressor(alpha = 0.01, fit_intercept=False, learning_rate='invscaling', loss='squared_loss', penalty='l2', tol=0.001)
sgd2.fit(X_train, y_train)
predict = sgd2.predict(X_test)
print(r2_score(y_test, predict))
print(mean_squared_error(y_test, predict))

0.83023929542099
1246279769.964481


## Lasso

In [0]:
from sklearn.linear_model import Lasso

In [21]:
lass = Lasso(max_iter=2000)

lass.fit(X_train, y_train)

## testando e avaliando

predict = lass.predict(X_test)
print(r2_score(y_test, predict))
print(mean_squared_error(y_test, predict))

0.8763878438949542
907485214.8938252


### Refinando hiperparâmetros

In [0]:
params = {'alpha': [0.01, 0.02, 0.05, 0.07, 1.0],
         'fit_intercept': [True, False],
         'normalize': [True, False],
         'max_iter': [3000],
         'tol': [1e-4, 1e-3, 1],
         'selection':  ['random', 'cyclic'],
        }

In [0]:
# from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(lass, params, scoring='neg_mean_squared_error', cv=5)
gs.fit(X_train, y_train)

In [0]:
print(gs.best_score_)
print(gs.best_params_)

-1427853697.6005933
{'alpha': 1.0, 'fit_intercept': True, 'max_iter': 3000, 'normalize': True, 'selection': 'random', 'tol': 0.0001}


### Testando novos hiperparâmetros

In [23]:
lass2 = Lasso(alpha = 1.0, fit_intercept=True, max_iter=3000, normalize=True, selection='random', tol=0.0001)
lass2.fit(X_train, y_train)
predict = lass2.predict(X_test)
print(r2_score(y_test, predict))
print(mean_squared_error(y_test, predict))

0.8802841040239788
878881244.4439807


## ElasticNet

In [0]:
from sklearn.linear_model import ElasticNet

In [24]:
en = ElasticNet()

en.fit(X_train, y_train)

## testando e avaliando

predict = en.predict(X_test)
print(r2_score(y_test, predict))
print(mean_squared_error(y_test, predict))

0.6291949126114433
2722225265.018186


### Refinando hiperparâmetros

In [0]:
params = {'alpha':[0.1, 0.5, 1.0, 1.5],
          'l1_ratio':[0.5, 0.2, 0.1, 0.8, 1.0],
          'fit_intercept':[True, False],
          'normalize':[True, False],
          'precompute':[True, False],
          'max_iter':[3000],
          'copy_X':[True, False],
          'tol':[0.0001],
          'positive':[True, False],
          'selection':['cyclic', 'random'], 
        }

In [0]:
# from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(en, params, scoring='neg_mean_squared_error', cv=5)
gs.fit(X_train, y_train)

In [0]:
print(gs.best_score_)
print(gs.best_params_)

0.8283745564564697
{'alpha': 0.1, 'copy_X': True, 'fit_intercept': False, 'l1_ratio': 0.8, 'max_iter': 3000, 'normalize': True, 'positive': False, 'precompute': False, 'selection': 'cyclic', 'tol': 0.0001}


### Testando novos hiperparâmetros

In [25]:
en2 = ElasticNet(alpha = 0.1, fit_intercept=False, l1_ratio = 0.8, max_iter=3000, normalize=True, positive=False, selection='cyclic', tol=0.0001)
en2.fit(X_train, y_train)
predict = en2.predict(X_test)
print(r2_score(y_test, predict))
print(mean_squared_error(y_test, predict))

0.8248585569454807
1285781876.922193


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 484266688084.08386, tolerance: 3772619776.7556
  positive)


# Validação cruzada dos modelos

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
scores_sgd2 = cross_val_score(sgd2, X, y, cv=10, scoring='r2')
scores_lass = cross_val_score(lass2, X, y, cv=10, scoring='r2')
scores_elastic = cross_val_score(en2, X, y, cv=10, scoring='r2')

In [49]:
print("R2 do SGD: " + str(np.mean(scores_sgd2)) + "    +/- " + str(round(np.std(scores_sgd2), 2)))
print("R2 do Lasso: " + str(np.mean(scores_lass))+ "    +/- " + str(round(np.std(scores_lass), 2)))
print("R2 do ElasticNet: " + str(np.mean(scores_elastic))+ "    +/- " + str(round(np.std(scores_elastic), 2)))

R2 do SGD: 0.850055265864777    +/- 0.06
R2 do Lasso: 0.8125677143214762    +/- 0.14
R2 do ElasticNet: 0.8458553953328913    +/- 0.06
